In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
demand = {}
random.seed(1) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(1) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(1) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(1) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(1) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs = dk.to_dict()
cols = sorted(demand.keys())

In [2]:
costs1=copy.deepcopy(costs)
costs2=copy.deepcopy(costs)
costs3=copy.deepcopy(costs)
for i in supply:
    mi=min(costs[i].values())
    # print(costs[i])
    # print(mi)
    for j in costs2[i]:
        costs2[i][j]-=mi
# print(costs2)
for i in demand :
    mi=10000
    for j in supply:
        if costs[j][i]<mi :
            mi=costs[j][i]
    for j in supply:
        costs3[j][i]=costs3[j][i]-mi
# print(costs3)

for i in demand:
    for j in supply:
        costs[j][i]= costs2[j][i]+costs3[j][i]


res = dict((k, defaultdict(int)) for k in costs)
g = {}
for x in supply:
    g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
for x in demand:
    g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])


# print(costs1)
flag=0
while g:

    d = {}
    # print(demand,supply)

    s = {}
    for x in supply:
        s[x] = 0
        for y in demand:
            s[x]=s[x] + costs[x][y]- costs[x][g[x][0]]

    for y in demand :
        s[y]=0
        for x in supply :
            s[y] =s[y] + costs[x][y]- costs[g[y][0]][y]
    t,f="S","D"
    rx = max(s, key=lambda n: s[n])
    if demand.get(rx)!=None :
        t,f = g[rx][0] , rx
    else :
        t,f = rx , g[rx][0]

    v= min(supply[t],demand[f])
    demand[f]-=v
    supply[t]-=v
    res[t][f]+=v

    if demand[f]==0:
        for k, n in supply.items():
            if n != 0:
                g[k].remove(f)
                # g1[k].remove(f)
        del g[f]
        # del g1[t]
        del demand[f]
    if supply[t]==0:
        for k, n in demand.items():
            if n != 0:
                g[k].remove(t)
                # g1[k].remove(f)
        del g[t]
        # del g1[f]
        del supply[t]


#print(demand,supply)

# print(costs1)
cost = 0
for g in sorted(costs1):
    # print (g, " ",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")
# print ("Total Cost = ", cost)

In [3]:
cost

258303.86665716072